In [25]:
import conllu
from io import open
import pickle 
import numpy as np

In [26]:
folder = "./UD_Ukrainian-IU/"
train = "uk_iu-ud-train.conllu"
test = "uk_iu-ud-test.conllu"

In [27]:
train_data = conllu.parse_incr(open(folder+train,'r',encoding = "utf-8"))

In [28]:
c3 = {} # триграммы
c2 = {} # биграммы
c1 = {} # монограммы
cT = {} # пары слово-тег
for token in train_data:
    sentence = [["*","*"]] + [["*","*"]] + [[token[i]["lemma"], token[i]["upostag"]]
        for i in range(len(token)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) 

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )

In [29]:
print (len(c3), len(c2), len(c1),len(cT))

# Save:
with open('cs_ukr.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)

2391 286 20 15233


In [30]:
with open('cs_ukr.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [31]:
tags = list(c1.keys())
tags.remove('*')
tags.remove('STOP')
k = len(tags) 

In [32]:

def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): # выдает возможные тэги в зависимости от положения слова в предложении (в начале предложения - *) 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))

['ADP', 'NOUN', 'ADJ', 'PROPN', 'VERB', 'PUNCT', 'CCONJ', 'ADV', 'PRON', 'PART', 'DET', 'SCONJ', 'NUM', 'X', 'AUX', 'INTJ', '_', 'SYM']


Алгоритм Витерби

In [33]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    print(y[1:])
    return y[1:]

Прогоняем алгоритм на текстовом корпусе

In [34]:
data_test = conllu.parse_incr(open(folder+test,'r',encoding = "utf-8"))
test_tags = [] # оригинальные теги корпуса
predict_tags = [] # предсказанные алгоритмом

for token in data_test:
    sentence=["*"] + [token[i]["lemma"] for i in range(len(token)) ] + ["STOP"]
    test_tags.append([token[i]["upostag"] for i in range(len(token)) ])
    predict_tags.append(viterbi(sentence))

['NOUN', 'ADJ', 'NOUN', 'PUNCT', 'PRON', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT']
['VERB', 'DET', 'NOUN', 'PROPN', 'PUNCT', 'NUM', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PRON', 'PART', 'VERB', 'PUNCT', 'PUNCT', 'ADV', 'PUNCT']
['PRON', 'VERB', 'ADP', 'NOUN', 'NOUN', 'PUNCT', 'SCONJ', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'DET', 'X', 'X', 'PUNCT', 'SCONJ', 'VERB', 'PUNCT']
['CCONJ', 'PRON', 'PART', 'PRON', 'AUX', 'NOUN', 'ADJ', 'PUNCT', 'SCONJ', 'ADP', 'NOUN', 'PRON', 'PART', 'VERB', 'PUNCT']
['VERB', 'PUNCT', 'INTJ', 'PUNCT', 'PUNCT', 'VERB', 'ADV', 'CCONJ', 'VERB', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT']
['ADP', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'ADJ', 'PART', 'ADP', 'NOUN', 'NOUN', 'ADP', 'PUNCT', 'NOUN', 'PUNCT', 'PUNCT', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADV', 'VERB', 'PUNCT', 'SCONJ', 'PART', 'VERB', 'PUNCT', 'VERB', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'SCONJ', 'PART', 'VERB', 'DET', 'PUNCT', 'SCONJ', 'PART', 'PUNCT', 'PRON', 'PART', 'VERB', 'PUNCT', 'PUNCT', 'PUNCT']
['CCONJ', 

['NOUN', 'PROPN', 'PUNCT', 'INTJ', 'PUNCT', 'INTJ', 'PUNCT', 'INTJ', 'PUNCT', 'INTJ', 'PUNCT', 'INTJ', 'PUNCT', 'INTJ', 'PUNCT', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'VERB', 'NOUN', 'ADJ', 'NOUN', 'ADV', 'ADP', 'NOUN', 'PROPN', 'PUNCT']
['CCONJ', 'PART', 'ADP', 'ADJ', 'NOUN', 'ADP', 'PROPN', 'VERB', 'ADJ', 'NOUN', 'PUNCT']
['VERB', 'NOUN', 'PUNCT', 'NOUN', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'PUNCT', 'PRON', 'ADV', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'SCONJ', 'ADJ', 'NOUN', 'X', 'PUNCT', 'X', 'PUNCT', 'X', 'CCONJ', 'DET', 'PROPN', 'PROPN', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'CCONJ', 'NOUN', 'ADJ', 'NOUN', 'DET', 'NOUN', 'NOUN', 'PUNCT']
['ADP', 'DET', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'PUNCT', 'X', 'PUNCT', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT']
['ADP', 'DET', 'NOUN', 'AUX', 'DET', 'ADJ', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'ADJ', 'ADJ', 'ADJ', 'NOUN', 'PUNCT', 'SCONJ', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'VERB', 'ADP', 'NOUN', 'CCONJ', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'ADJ', 'ADJ', 'ADJ'

['ADV', 'PRON', 'VERB', 'ADP', 'NOUN', 'X', 'PUNCT', 'X', 'PUNCT', 'ADJ', 'NOUN', 'PUNCT', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'ADV', 'PART', 'VERB', 'PUNCT']
['ADV', 'PART', 'NOUN', 'ADJ', 'VERB', 'NOUN', 'PUNCT', 'CCONJ', 'SCONJ', 'PART', 'VERB', 'PUNCT', 'ADP', 'NOUN', 'ADP', 'X', 'X', 'PUNCT', 'VERB', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'ADV', 'ADJ', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'CCONJ', 'ADP', 'NOUN', 'NOUN', 'PUNCT']
['VERB', 'ADP', 'PROPN', 'CCONJ', 'PROPN', 'PUNCT', 'ADV', 'PRON', 'ADP', 'PRON', 'PUNCT', 'CCONJ', 'ADP', 'PRON', 'VERB', 'PUNCT', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'X', 'PUNCT', 'PUNCT', 'SCONJ', 'PRON', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'X', 'CCONJ', 'DET', 'DET', 'NOUN', 'ADJ', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'NOUN', 'PUNCT', 'INTJ', 'PUNCT', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT']
['ADP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'PUNC

['ADV', 'PRON', 'VERB', 'ADP', 'NOUN', 'CCONJ', 'VERB', 'ADP', 'PUNCT', 'PROPN', 'PUNCT', 'PUNCT', 'PART', 'ADV', 'ADP', 'NOUN', 'PRON', 'VERB', 'PUNCT', 'PART', 'PRON', 'PUNCT', 'ADP', 'NOUN', 'ADP', 'PRON', 'VERB', 'PUNCT', 'SCONJ', 'ADV', '_', 'VERB', 'ADV', 'ADV', 'PUNCT']
['ADV', 'NOUN', 'VERB', 'PUNCT', 'NOUN', 'VERB', 'PUNCT', 'DET', 'VERB', 'PUNCT']
['NOUN', 'VERB', 'PUNCT', 'SCONJ', 'ADV', 'PRON', 'VERB', 'DET', 'NOUN', 'PUNCT']
['NOUN', 'VERB', 'ADP', 'NOUN', 'PUNCT']
['CCONJ', 'PRON', 'AUX', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT']
['VERB', 'NOUN', 'PUNCT', 'INTJ', 'PUNCT']
['ADV', 'ADV', 'AUX', 'VERB', 'NOUN', 'PUNCT']
['PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'CCONJ', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'SCONJ', 'ADV', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT']
['CCONJ', 'NOUN', 'AUX', 'ADJ', 'NOUN', 'PUNCT']
['NOUN', 'PUNCT', 'ADJ', 'PUNCT']
['ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'VERB', 'PUNCT', 'ADV', 'PUNCT', 'PRON', 'ADP', 'NOUN', 'A

['DET', 'PRON', 'PART', 'VERB', 'VERB', 'ADP', 'ADJ', 'NOUN', 'PUNCT']
['ADP', 'NOUN', 'PUNCT', 'DET', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'ADJ', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'ADP', 'PRON', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'PUNCT', 'ADP', 'NOUN', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'DET', 'NOUN', 'NOUN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'VERB', 'ADP', 'NOUN', 'ADP', 'NOUN', 'DET', 'NOUN', 'NOUN', 'PUNCT', 'SCONJ', 'ADP', 'NOUN', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'SYM', 'SYM']
['NOUN', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'X', 'X', 'ADJ', 'VERB', 'ADP', 'NUM', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PRON', 'NOUN', 'PUNCT']
['NOUN', 'ADP', 'PROPN', 'CCONJ', 'ADJ', 'ADJ', '

...

['NOUN', 'AUX', 'NOUN', 'NUM', 'PUNCT', 'NUM', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'CCONJ', 'PART', 'VERB', 'PUNCT', 'ADP', 'PROPN', 'PUNCT']
['ADP', 'PRON', 'PART', 'AUX', 'ADJ', 'PUNCT', 'SCONJ', 'VERB', 'VERB', 'NOUN', 'NOUN', 'PUNCT']
['ADP', 'DET', 'PART', 'NOUN', 'ADV', 'VERB', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'VERB', 'PUNCT', 'CCONJ', 'PART', 'ADV', 'ADJ', 'NOUN', 'ADP', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'PUNCT']
['NOUN', 'ADJ', 'NOUN', 'NUM', 'PUNCT', 'NUM', 'VERB', 'ADP', 'ADJ', 'NOUN', 'ADV', 'VERB', 'NOUN', 'PUNCT', 'PUNCT', 'CCONJ', 'NUM', 'NOUN', 'DET', 'PART', 'NOUN', 'NOUN', 'ADV', 'VERB', 'ADP', 'NOUN', 'PUNCT']
['NUM', 'PUNCT', 'NUM', 'VERB', 'ADJ', 'ADP', 'PROPN', 'CCONJ', 'ADJ', 'ADP', 'NOUN', 'PUNCT', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'PUNCT', 'X', 'PUNCT', 'PUNCT', 'NOUN', 'ADP', 'PRON', 'PUNCT', 'SCONJ', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'PUNCT', 'PUNCT']
['ADV', 'PUNCT', 'DET', 'NOUN', 'NOUN', '_', 'VERB', 'ADV', 'ADV', 'ADJ', 'PUNCT', 'SCONJ', 

['VERB', 'ADV', 'VERB', 'PUNCT']
['ADV', 'VERB', 'ADP', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'PUNCT', 'SCONJ', 'ADP', 'NOUN', 'NOUN', 'PUNCT', 'SCONJ', 'ADP', 'PRON', 'NOUN', 'PUNCT', 'ADV', 'ADV', 'PUNCT']
['VERB', 'ADP', 'NOUN', 'PUNCT', 'ADV', 'PUNCT', 'SCONJ', 'ADJ', 'NOUN', 'NOUN', 'PROPN', 'PROPN', 'VERB', 'NOUN', 'PUNCT', 'NUM', 'PUNCT', 'NUM', 'NOUN', 'ADJ', 'NOUN', 'PUNCT', 'PUNCT', 'NOUN', 'PROPN', 'PUNCT', 'DET', 'DET', 'NOUN', 'PRON', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'VERB', 'PUNCT', 'SCONJ', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'ADJ', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'NUM', 'NOUN', 'NOUN', 'PUNCT', 'PUNCT']
['ADV', 'PART', 'VERB', 'PUNCT', 'PART', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'VERB', 'VERB', 'NOUN', 'PUNCT']
['ADV', 'ADP', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'VERB', 'ADP', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PROPN', 'VERB', 'ADJ', 'ADJ', 'NOUN', 'PUNCT']
['ADP', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'VERB', 'NOUN', 'NOUN', 'VERB', 'NOUN', 'PUNCT']
['ADV', 'NOUN'

['SCONJ', 'NOUN', 'PUNCT', 'DET', 'VERB', 'PUNCT', 'SCONJ', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'PUNCT', 'PUNCT', 'VERB', 'AUX', 'NOUN', 'PUNCT']
['NOUN', 'PART', 'VERB', 'AUX', 'NOUN', 'PUNCT', 'SCONJ', 'AUX', 'PRON', 'PART', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'PUNCT']
['CCONJ', 'NOUN', 'VERB', 'PUNCT']
['PUNCT', 'ADJ', 'NOUN', 'PUNCT', 'SCONJ', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'DET', 'NOUN', 'CCONJ', 'ADV', 'PART', 'VERB', 'ADP', 'PRON', 'VERB', 'PUNCT']
['DET', 'PUNCT', 'SCONJ', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'PART', 'PART', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'CCONJ', 'PART', 'VERB', 'ADV', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'PUNCT']
['PART', 'ADV', 'ADJ', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'VERB', 'PUNCT', 'CCONJ', 'ADJ', 'PUNCT', 'NOUN', 'PUNCT']
['DET', 'NOUN', 'ADV', 'CCONJ', 'ADV', 'ADP', 'ADJ', 'NOUN', 'VERB', 'ADP', 'PRON', 'PUNCT', 'SCONJ', 'ADP', 'NOUN', 'PUNCT', 'PUNCT', 'SCONJ', 'ADP', 'NUM', 'NOUN', 'PUNCT']
['CCONJ', 'PRON', 'ADV', 'PART', 'VERB', 'PUNCT', 'SCO

['ADJ', 'NOUN', 'PUNCT', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADV', 'VERB', 'NOUN', 'CCONJ', 'VERB', 'ADP', 'NUM', 'ADP', 'NUM', 'NOUN', 'PUNCT', 'SCONJ', 'ADP', 'ADJ', 'NOUN', 'VERB', 'NOUN', 'PUNCT']
['ADP', 'NOUN', 'SYM', 'PUNCT', 'ADP', 'PRON', 'VERB', 'ADJ', 'NOUN', 'PUNCT']
['PRON', 'VERB', 'CCONJ', 'VERB', 'PUNCT']
['VERB', 'PUNCT', 'SCONJ', 'NOUN', 'PRON', 'VERB', 'PUNCT', 'SCONJ', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'DET', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PUNCT']
['ADP', 'ADJ', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'PUNCT']
['PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'VERB', 'PUNCT', 'CCONJ', 'ADV', 'PRON', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'CCONJ', 'ADV', 'PART', 'VERB', 'ADV', 'ADP', 'NOUN', 'PUNCT']
['CCONJ', 'ADP', 'DET', 'NOUN', 'NOUN', 'CCONJ', 'VERB', 'NOUN', 'ADP', 'X', 'X', 'PUNCT']
['NOUN', 'VERB', 'ADV', 'ADP', 'DET', 'NOUN', 'PUNCT', 'CCONJ', 'ADV', 'ADP', 'ADV', 'ADJ', 'NOUN', 'PUNCT', 'ADP', 'ADJ', 'NOUN', 'DET', 'VERB', 'ADV', 'NOUN', 'PUNCT', 'AUX', 'ADV', 'ADV', 'PUNCT']
['NUM', 'NOU

['NOUN', 'VERB', 'ADP', 'NOUN', 'ADJ', 'NOUN', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'AUX', 'ADV', 'SYM', 'CCONJ', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PROPN', 'PUNCT', 'PUNCT']
['PUNCT', 'ADV', 'ADV', 'PUNCT', 'SCONJ', 'DET', 'ADJ', 'NOUN', 'ADV', 'VERB', 'ADP', 'PRON', 'PUNCT', 'DET', 'NOUN', 'VERB', 'ADP', 'PROPN', 'PUNCT']
['SYM', 'NOUN', 'ADP', 'NOUN', 'CCONJ', 'NOUN', 'NOUN', 'VERB', 'ADP', 'NOUN', 'VERB', 'NOUN', 'CCONJ', 'NOUN', 'ADJ', 'NOUN', 'PUNCT', 'SCONJ', 'NOUN', 'VERB', 'ADP', 'NOUN', 'PUNCT', 'NOUN', 'CCONJ', 'PUNCT', 'NOUN', 'PUNCT', 'ADP', 'NOUN', 'PUNCT', 'CCONJ', 'PART', 'ADP', 'NOUN', 'PUNCT']
['ADJ', 'NOUN', 'ADV', 'VERB', 'PUNCT', 'SCONJ', 'PRON', 'AUX', 'DET', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'DET', 'VERB', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'PUNCT', 'PUNCT', 'PROPN', 'PROPN', 'PROPN', 'PUNCT']
['PUNCT', 'NOUN', 'VERB', 'NUM', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'PUNCT', 'ADJ', 'ADP', 'ADJ', 'NOUN', 'PUNCT', 'DET', 

Считаем ошибку

In [21]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))

Error =  0.11879883257196387
